In [205]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

## Men's Data

In [206]:
# Load data into Pandas dataframes
mRegDetail = pd.read_csv('data/men data/MRegularSeasonDetailedResults.csv')
mNames = pd.read_csv('data/men data/MTeamSpellings.csv')

In [207]:
display(mRegDetail.columns.values)

array(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore',
       'WLoc', 'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA',
       'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA',
       'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO',
       'LStl', 'LBlk', 'LPF'], dtype=object)

In [208]:
# Split regular season detailed results into dataframes focused on outcome for one team
mRegWinners = pd.DataFrame()
mRegLossers = pd.DataFrame()

# Establish new columns for that includes stats for one team
columns = ['Season', 'TeamID', 'Score', 'OppScore',
       'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA',
       'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OppFGM', 'OppFGA',
       'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO',
       'OppStl', 'OppBlk', 'OppPF']

In [209]:
# Split winners from regular season
mRegWinners[columns] = mRegDetail[['Season', 'WTeamID', 'WScore', 'LScore',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA',
       'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA',
       'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO',
       'LStl', 'LBlk', 'LPF']]

# Add wins and losses columns
mRegWinners['Wins'] = 1
mRegWinners['Losses'] = 0

In [210]:
# Split lossers from regular season
mRegLossers[columns] = mRegDetail[['Season', 'LTeamID', 'LScore', 'WScore',
       'NumOT', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA',
       'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'WFGM', 'WFGA',
       'WFGM3', 'WFGA3', 'WFTM', 'WFTA','WOR', 'WDR', 'WAst', 'WTO',
       'WStl', 'WBlk', 'WPF']]

# Add wins and losses columns
mRegLossers['Wins'] = 0
mRegLossers['Losses'] = 1

In [211]:
# Combine all games into one dataframe
mAllRegDetail = pd.concat([mRegWinners, mRegLossers])

In [212]:
# Sum stats for each season for each team for regular season
mRegSeasonDetail = mAllRegDetail.groupby(['Season', 'TeamID']).sum(numeric_only=True)
mRegSeasonDetail['NumGames'] = mRegSeasonDetail['Wins'] + mRegSeasonDetail['Losses']

In [213]:
mRegSeasonDetail

Score  OppScore  NumOT  FGM   FGA  FGM3  FGA3  FTM  FTA   OR  \
Season TeamID                                                                 
2003   1102     1603      1596      0  536  1114   219   583  312  479  117   
       1103     2127      2110      8  733  1508   147   434  514  698  264   
       1104     1940      1820      1  673  1601   178   556  416  586  380   
       1105     1866      1993      4  634  1602   197   540  401  568  351   
       1106     1781      1785      1  656  1548   171   494  298  461  344   
...              ...       ...    ...  ...   ...   ...   ...  ...  ...  ...   
2025   1476     1964      2056      0  676  1561   250   701  362  499  204   
       1477     1995      2321      1  713  1714   260   828  309  480  246   
       1478     2091      2356      7  718  1609   221   669  434  605  216   
       1479     1842      2009      3  635  1507   196   549  376  466  174   
       1480     2033      2365      0  760  1779   159   536  354  513  245   

               ...  OppOR  OppDR  OppAst  OppTO  OppStl  OppBlk  OppPF  Wins  \
Season TeamID  ...                                                             
2003   1102    ...    269    564     256    363     152      44    514    12   
       1103    ...    325    595     418    414     173      77    606    13   
       1104    ...    305    634     327    388     155      89    539    17   
       1105    ...    343    686     411    489     244     109    496     7   
       1106    ...    317    626     330    422     246      89    452    13   
...            ...    ...    ...     ...    ...     ...     ...    ...   ...   
2025   1476    ...    258    644     299    289     196     111    457    13   
       1477    ...    311    728     491    404     286     119    476     5   
       1478    ...    302    671     464    318     215      84    512     7   
       1479    ...    233    662     389    351     161      79    472    12   
       1480    ...    234    754     471    326     206      93    461     5   

               Losses  NumGames  
Season TeamID                    
2003   1102        16        28  
       1103        14        27  
       1104        11        28  
       1105        19        26  
       1106        15        28  
...               ...       ...  
2025   1476        16        29  
       1477        26        31  
       1478        22        29  
       1479        16        28  
       1480        25        30  

[7981 rows x 32 columns]

In [214]:
# Create ___ per game stat for each boxscore stat
mRegSeasonFeatures = pd.DataFrame()

stats = ['Score', 'OppScore', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA',
       'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OppFGM', 'OppFGA',
       'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO',
       'OppStl', 'OppBlk', 'OppPF']

for col in stats:
    mRegSeasonFeatures[col + '_PerGame'] = mRegSeasonDetail[col] / mRegSeasonDetail['NumGames']

In [215]:
mRegSeasonFeatures

Score_PerGame  OppScore_PerGame  NumOT_PerGame  FGM_PerGame  \
Season TeamID                                                                
2003   1102        57.250000         57.000000       0.000000    19.142857   
       1103        78.777778         78.148148       0.296296    27.148148   
       1104        69.285714         65.000000       0.035714    24.035714   
       1105        71.769231         76.653846       0.153846    24.384615   
       1106        63.607143         63.750000       0.035714    23.428571   
...                      ...               ...            ...          ...   
2025   1476        67.724138         70.896552       0.000000    23.310345   
       1477        64.354839         74.870968       0.032258    23.000000   
       1478        72.103448         81.241379       0.241379    24.758621   
       1479        65.785714         71.750000       0.107143    22.678571   
       1480        67.766667         78.833333       0.000000    25.333333   

               FGA_PerGame  FGM3_PerGame  FGA3_PerGame  FTM_PerGame  \
Season TeamID                                                         
2003   1102      39.785714      7.821429     20.821429    11.142857   
       1103      55.851852      5.444444     16.074074    19.037037   
       1104      57.178571      6.357143     19.857143    14.857143   
       1105      61.615385      7.576923     20.769231    15.423077   
       1106      55.285714      6.107143     17.642857    10.642857   
...                    ...           ...           ...          ...   
2025   1476      53.827586      8.620690     24.172414    12.482759   
       1477      55.290323      8.387097     26.709677     9.967742   
       1478      55.482759      7.620690     23.068966    14.965517   
       1479      53.821429      7.000000     19.607143    13.428571   
       1480      59.300000      5.300000     17.866667    11.800000   

               FTA_PerGame  OR_PerGame  ...  OppFGA3_PerGame  OppFTM_PerGame  \
Season TeamID                           ...                                    
2003   1102      17.107143    4.178571  ...        12.428571       13.678571   
       1103      25.851852    9.777778  ...        18.370370       15.925926   
       1104      20.928571   13.571429  ...        19.142857       12.142857   
       1105      21.846154   13.500000  ...        17.538462       16.384615   
       1106      16.464286   12.285714  ...        15.214286       15.535714   
...                    ...         ...  ...              ...             ...   
2025   1476      17.206897    7.034483  ...        18.827586       13.689655   
       1477      15.483871    7.935484  ...        19.483871       14.225806   
       1478      20.862069    7.448276  ...        24.689655       16.206897   
       1479      16.642857    6.214286  ...        20.821429       14.250000   
       1480      17.100000    8.166667  ...        24.933333       14.866667   

               OppFTA_PerGame  OppOR_PerGame  OppDR_PerGame  OppAst_PerGame  \
Season TeamID                                                                 
2003   1102         19.250000       9.607143      20.142857        9.142857   
       1103         22.148148      12.037037      22.037037       15.481481   
       1104         17.142857      10.892857      22.642857       11.678571   
       1105         24.500000      13.192308      26.384615       15.807692   
       1106         21.964286      11.321429      22.357143       11.785714   
...                       ...            ...            ...             ...   
2025   1476         18.655172       8.896552      22.206897       10.310345   
       1477         20.548387      10.032258      23.483871       15.838710   
       1478         22.206897      10.413793      23.137931       16.000000   
       1479         20.000000       8.321429      23.642857       13.892857   
       1480         20.666667       7.800000      25.133333       15.700000   

       

In [216]:
# Create additional features
mRegSeasonFeatures['W/L'] = mRegSeasonDetail['Wins'] / mRegSeasonDetail['Losses'] # Win/Loss ratio
mRegSeasonFeatures['MOV'] = (mRegSeasonDetail['Score'] - mRegSeasonDetail['OppScore']) / mRegSeasonDetail['NumGames'] # Margin of victory
mRegSeasonFeatures['TORatio'] = mRegSeasonFeatures['TO_PerGame'] / mRegSeasonFeatures['OppTO_PerGame'] # Turnover ratio